In [16]:
import dask.dataframe as dd
import pandas as pd

from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
df = pd.read_csv('../datasets/cleaned/merged_by_asin.csv')

In [9]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [10]:
df.sort_values(by=['label', 'asin'], ignore_index=True, inplace=True)
df

text     category  \
0       I have a badger 5 it is leaking water at botto...   appliances   
1       I have a 9 year old Badger 1 that needs replac...   appliances   
2       burner plates for electric glass cooktops? Can...   appliances   
3       burner plates for electric glass cooktops? Can...   appliances   
4       How many test kits are there in a box. How man...   appliances   
...                                                   ...          ...   
182017  will this work on 2ds? Yes all 3ds games will ...  video_games   
182018  Is $ 19.99 the price for the game itself? Or i...  video_games   
182019  Will this game spoil anything for someone who ...  video_games   
182020  will this work on imacs new update it will wor...  video_games   
182021  how many gig on the hard drive It is a 500gb h...  video_games   

              asin  label  
0       B00004U9JO      0  
1       B00004U9JP      0  
2       B00004W4UJ      0  
3       B00004W4UK      0  
4       B00005AUHX      0  
...            ...    ...  
182017  B00JM3R6M6     20  
182018  B00JQHU9RC     20  
182019  B00K2O4O6U     20  
182020  B00K4S9C9I     20  
182021  B00KAI3KW2     20  

[182022 rows x 4 columns]

In [18]:
def split_data_by_category(data, n_classes=21, col='label'):
    return [data.groupby(col).get_group(cat) for cat in range(n_classes)]    

In [19]:
dfs = split_data_by_category(df)

In [32]:
def calculate_matrix(data):
    

(1153, 5)

In [35]:
test_df = dfs[0].copy()

In [37]:
QUESTION = "Does the badger come with a dishwasher hook up?"

In [38]:
Text=test_df['text'].tolist()
Text.insert(0, QUESTION)
tfidf=text.TfidfVectorizer(input=Text,stop_words="english")
matrix=tfidf.fit_transform(Text)

In [39]:
Text

['Does the badger come with a dishwasher hook up?',
 "I have a badger 5 it is leaking water at bottom near reset bottom plus does a new one came with a ss ring for inside the sink If your garbage disposal is leaking out the bottom near the reset button, you most likely have a leak in the metal plate that is located directly under the blades. This plate is designed to protect the electrically powered parts of the motor (the rotor, the shaft, etc.) from moisture, but it is rather thin and is only made of galvanized steel. After a year or two, especially if you use highly acidic cleaners which you rinse down your sink, the zinc layer will corrode and expose the steel, which will then also start to corrode, until something hard eventually pokes some kind of hole through the plate somewhere (could be somewhere very small), signaling the end of the machine's service life. The plate is not meant to be repaired, only replaced along with the rest of the unit. That water is getting through to yo

In [40]:
matrix.shape

(1154, 16747)

In [41]:
sim_unigram=cosine_similarity(matrix)

In [42]:
sim_unigram

array([[1.        , 0.42263883, 0.55672195, ..., 0.00873714, 0.01454705,
        0.00262045],
       [0.42263883, 1.        , 0.47060033, ..., 0.04263192, 0.02468046,
        0.02904963],
       [0.55672195, 0.47060033, 1.        , ..., 0.02645113, 0.02095488,
        0.01481082],
       ...,
       [0.00873714, 0.04263192, 0.02645113, ..., 1.        , 0.03356766,
        0.24719653],
       [0.01454705, 0.02468046, 0.02095488, ..., 0.03356766, 1.        ,
        0.01855404],
       [0.00262045, 0.02904963, 0.01481082, ..., 0.24719653, 0.01855404,
        1.        ]])

In [31]:
sim_unigram.shape

(1153, 1153)

In [45]:
sim_unigram[0]

array([1.        , 0.42263883, 0.55672195, ..., 0.00873714, 0.01454705,
       0.00262045])

In [29]:
def get_similar(x):
    return ",".join(dfs[0]['asin'].loc[x.argsort()[-5:-1]])

C:\Users\mriva\.conda\envs\torch\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
test_df

text    category  \
0     I have a badger 5 it is leaking water at botto...  appliances   
1     I have a 9 year old Badger 1 that needs replac...  appliances   
2     burner plates for electric glass cooktops? Can...  appliances   
3     burner plates for electric glass cooktops? Can...  appliances   
4     How many test kits are there in a box. How man...  appliances   
...                                                 ...         ...   
1148  what are the dimensions? Overall Width: 27" Ov...  appliances   
1149  How many pieces of clothes can you put in the ...  appliances   
1150  What is the name of the manufacturer? (waterfa...  appliances   
1151  Can two (or more) Lyric Thermostats coexist in...  appliances   
1152  has anyone used it with a firgidaire refrigera...  appliances   

            asin  label                                      unigram  
0     B00004U9JO      0  B001B4E0VY,B000DZGN7Q,B000BQOWES,B00004U9JP  
1     B00004U9JP      0  B001B4E0VY,B000BQ7WE0,B000BQOWES,B00004U9JO  
2     B00004W4UJ      0  B001AAEG7M,B0011YJE7E,B005TLMBHK,B00004W4UJ  
3     B00004W4UK      0  B001AAEG7M,B0011YJE7E,B005TLMBHK,B00004W4UJ  
4     B00005AUHX      0  B007RE55G6,B005NIBEYU,B000LO8SIQ,B00B9DZLAG  
...          ...    ...                                          ...  
1148  B00J39MKMA      0  B008BVX4QC,B0038ZS4I4,B008KTIGZ4,B00GXCPRYI  
1149  B00JJZBHK4      0  B006DTBTOI,B005GM942C,B0007MEW14,B00HZRJVCO  
1150  B00KJ07SEM      0  B001BOTNCU,B004H3XVKM,B000AST3AK,B0093NFXJI  
1151  B00KX3M1LA      0  B005EJ7YO2,B004MH0S40,B000TD1JVC,B005Y29KL4  
1152  B00L07RWJE      0  B000UW3RK8,B000KI3O1O,B0093NFXJI,B001BOTNCU  

[1153 rows x 5 columns]

In [46]:
get_similar(sim_unigram[0])

'B006YGEKIW,B000BQW5OC,B00004U9JP,B00004W4UJ'